In [1]:
pip install websockets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install nest_asyncio

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# cliente.py
import asyncio
import threading
import tkinter as tk
import websockets

class ChatClienteGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Cliente de Chat WebSocket")
        self.nickname = ""
        self.ws = None

        # Parte superior: ingreso del nombre
        self.frame_top = tk.Frame(self.root)
        self.label_nombre = tk.Label(self.frame_top, text="Apodo:")
        self.entry_nombre = tk.Entry(self.frame_top)
        self.btn_conectar = tk.Button(self.frame_top, text="Conectar", command=self.iniciar_conexion)

        self.label_nombre.pack(side=tk.LEFT)
        self.entry_nombre.pack(side=tk.LEFT, padx=5)
        self.btn_conectar.pack(side=tk.LEFT)
        self.frame_top.pack(pady=10)

        # Área de mensajes
        self.chat_area = tk.Text(self.root, height=20, width=50, state=tk.DISABLED)
        self.chat_area.pack(padx=10, pady=10)

        # Parte inferior: envío de mensajes
        self.frame_bottom = tk.Frame(self.root)
        self.entry_mensaje = tk.Entry(self.frame_bottom, width=40)
        self.btn_enviar = tk.Button(self.frame_bottom, text="Enviar", command=self.enviar_mensaje)

        self.entry_mensaje.pack(side=tk.LEFT, padx=5)
        self.btn_enviar.pack(side=tk.LEFT)
        self.frame_bottom.pack(pady=10)

        self.loop = asyncio.new_event_loop()
        threading.Thread(target=self.loop.run_forever, daemon=True).start()

        self.root.protocol("WM_DELETE_WINDOW", self.cerrar_ventana)
        self.root.mainloop()

    def iniciar_conexion(self):
        self.nickname = self.entry_nombre.get().strip()
        if self.nickname:
            self.agregar_mensaje("Conectando al servidor...")
            asyncio.run_coroutine_threadsafe(self.conectar(), self.loop)

    async def conectar(self):
        try:
            self.ws = await websockets.connect("ws://localhost:8765")
            self.agregar_mensaje("Conectado al servidor.")
            await self.ws.send(f"{self.nickname} se ha unido al chat.")
            asyncio.run_coroutine_threadsafe(self.recibir_mensajes(), self.loop)
        except Exception as e:
            self.agregar_mensaje(f"Error de conexión: {e}")

    async def recibir_mensajes(self):
        try:
            async for mensaje in self.ws:
                self.agregar_mensaje(mensaje)
        except websockets.exceptions.ConnectionClosed:
            self.agregar_mensaje("Conexión cerrada.")

    def enviar_mensaje(self):
        mensaje = self.entry_mensaje.get().strip()
        if self.ws and mensaje:
            mensaje_completo = f"{self.nickname}: {mensaje}"
            # Enviar al servidor
            asyncio.run_coroutine_threadsafe(
                self.ws.send(mensaje_completo),
                self.loop
            )
            # Mostrar inmediatamente en la interfaz del propio usuario
            self.agregar_mensaje(f"Tú: {mensaje}")
            self.entry_mensaje.delete(0, tk.END)


    def agregar_mensaje(self, mensaje):
        self.chat_area.config(state=tk.NORMAL)
        self.chat_area.insert(tk.END, mensaje + "\n")
        self.chat_area.config(state=tk.DISABLED)
        self.chat_area.see(tk.END)

    def cerrar_ventana(self):
        if self.ws:
            asyncio.run_coroutine_threadsafe(self.ws.close(), self.loop)
        self.root.destroy()

if __name__ == "__main__":
    ChatClienteGUI()
